## 1. What are the typical sizes/dimensions of these bounding boxes?

In [9]:
import json
import os
import pprint

In [16]:
with open("../updated_images/coco/output.json", 'r') as f:
    json_data = json.load(f)


In [27]:
heights = []
widths = []
for n, ann in enumerate(json_data):
    heights.append(json_data["annotations"][n]['bndbox']["xmax"] - json_data["annotations"][n]['bndbox']["xmin"] )
    widths.append(json_data["annotations"][n]['bndbox']["ymax"] - json_data["annotations"][n]['bndbox']["ymin"] )

    # Calculate the average of the height and width data
avg_height = sum(heights) / len(heights)
avg_width = sum(widths) / len(widths)

print(f'Average height: {avg_height:.2f}')
print(f'Average height: {avg_width:.2f}')

Average height: 192.67
Average height: 51.00


## 2. What are the typical locations of these boxes relative to the image space?

In [28]:
import cv2
import numpy as np


def find_average_location(json_data):
    total_center_x = 0
    total_center_y = 0
    total_boxes = 0
    for n, ann in enumerate(json_data):
        min_x = json_data["annotations"][n]['bndbox']["xmin"]
        min_y = json_data["annotations"][n]['bndbox']["ymin"]
        max_x = json_data["annotations"][n]['bndbox']["xmax"]
        max_y = json_data["annotations"][n]['bndbox']["ymax"]

        center_x = (min_x + max_x) // 2
        center_y = (min_y + max_y) // 2

        total_center_x += center_x
        total_center_y += center_y
        total_boxes += 1

    average_center_x = total_center_x / total_boxes
    average_center_y = total_center_y / total_boxes

    return average_center_x, average_center_y





In [31]:
average_center_x, average_center_y = find_average_location(json_data)

print(f"Average center location, x,y format: ({average_center_x}, {average_center_y})")

Average center location, x,y format: (515.3333333333334, 387.0)


In [33]:
def find_average_image(json_data):
    total_center_h = 0
    total_center_w = 0
    total_boxes = 0
    for n, ann in enumerate(json_data):
        h = json_data["images"][n]["height"]
        w = json_data["images"][n]["width"]

        total_center_h += h
        total_center_w += w
        total_boxes += 1

    average_center_h = total_center_h / total_boxes
    average_center_w = total_center_w / total_boxes

    return average_center_h, average_center_w


In [35]:
average_center_h, average_center_w = find_average_image(json_data)

print(f"Average image size, h,w format: ({average_center_h}, {average_center_w})")

Average image size, h,w format: (578.6666666666666, 873.0)


## 3. What potential issues could happen with license plate detection? 

Some issues that can happen with license plate detection go around that the model itself must be able to bound license plates then OCR the results. Lots of issues can happen with that, such as the difference in night vs day, cloudy, foggy, sun rays, shade, speed, etc. Then some license plate covers have words on it themselves, which means OCR could pick out the wrong word or words nearby.

To try and remedy this, obviously plenty of data can help, as well as potentially limiting the pixels to only grab what is needed, added noise and removing color can actually be beneficial, potentially, and making sure a human in the loop here will be evauluating what is needed. 

## 4. Could manipulating the image improve the models performance? Would decrease?

Manipulating the image could improve the models performance if there were some color issues that seemed to mask what was being looked at, maybe changine the colors to black and white would help hide fog or sun rays in an image that could distort what was actually attempting to be detected. Of course its also possible that the manipulations of the image manipulate teh desired object to much to the point where it is no longer able to be detected. Changing the image should be handled carefully in order to figure out what would generalize a model for the real world the best.